In [1]:
import numpy as np
import os
import pandas as pd

# find target path for data files, assuming the notebook is in the right place
data_path = _dh[0]
assert data_path.endswith(os.path.join('em-2020','data'))

output_path = os.path.join(data_path, 'combined_dataset_raw.csv')

In [2]:
country_groups = pd.read_excel(os.path.join(data_path, 'CLASS.xls'), sheet_name='Groups')
country_group_map = {k: set(v['CountryCode']) for k, v in country_groups.groupby('GroupCode')}
all_countries = country_group_map['WLD']

In [3]:
wdi_path = os.path.join(data_path, 'WDI_reshaped.csv')
wdi_data = pd.read_csv(wdi_path).set_index(['Country Code', 'Year'])
is_country = wdi_data.index.get_level_values('Country Code').isin(all_countries)

wipo_path = os.path.join(data_path, 'WIPO_reshaped.csv')
wipo_data = pd.read_csv(wipo_path).set_index(['Country Code', 'Year'])
assert set(all_countries).issuperset(wipo_data.index.levels[0])

chinn_ito_path = os.path.join(data_path, 'ChinnIto_reshaped.csv')
chinn_ito_data = pd.read_csv(chinn_ito_path).set_index(['Country Code', 'Year'])
assert set(all_countries).issuperset(chinn_ito_data.index.levels[0])

## Copy over raw data/simple transformations:

In [4]:
combined = pd.DataFrame()
combined['ENI'] = 1. / wdi_data['EG.GDP.PUSE.KO.PP.KD']
combined['POP'] = wdi_data['EN.POP.DNST']
combined['URB'] = wdi_data['SP.URB.TOTL.IN.ZS']
combined['AFL'] = wdi_data['NY.GDP.PCAP.PP.KD']

In [5]:
combined.xs('FRA', level='Country Code', drop_level=False).tail()

ENI         POP     URB           AFL
Country Code Year                                           
FRA          2015  0.08477  121.536704  79.655  43553.399292
             2016      NaN  121.857825  79.917  43914.481647
             2017      NaN  122.114008  80.180  44826.507474
             2018      NaN  122.299437  80.444  45561.001768
             2019      NaN         NaN  80.709  46183.521790

## Combine WB WDI and WIPO patent data

In [6]:
wb_patents = wdi_data[['IP.PAT.RESD', 'IP.PAT.NRES']].sum(axis='columns')
wipo_patents = wipo_data['WIPO patents']

patent_df = pd.DataFrame({'WB': wb_patents, 'WIPO': wipo_patents})
patent_df.replace(0., np.nan, inplace=True)
missing = patent_df.isnull().any(axis=1) 
mismatch = ~missing & (patent_df['WB'] != patent_df['WIPO'])

In [7]:
patent_df.loc[mismatch]

,,WB,WIPO
Country Code,Year,,


In [8]:
patent_df.loc[~patent_df['WB'].isnull() 
              & patent_df['WIPO'].isnull()
             ].drop('WLD', level='Country Code')

WB  WIPO
Country Code Year             
COD          1980  103.0   NaN
             1981   82.0   NaN
             1982   99.0   NaN
             1983   72.0   NaN
             1984   94.0   NaN

In [9]:
patent_df.loc[~patent_df['WIPO'].isnull() 
              & patent_df['WB'].isnull()
             ]

WB   WIPO
Country Code Year           
ALB          1993 NaN   16.0
             1994 NaN    8.0
             1995 NaN    5.0
             1996 NaN    5.0
             1998 NaN   21.0
...                ..    ...
SYR          2016 NaN  112.0
YEM          2008 NaN   85.0
             2009 NaN   72.0
ZWE          2013 NaN   23.0
             2014 NaN   27.0

[165 rows x 2 columns]

## Convert patent count to per capita measure

In [10]:
patent_count = patent_df['WB'].combine_first(patent_df['WIPO'])
combined['TI'] = patent_count / wdi_data['SP.POP.TOTL'] * 1e5

In [11]:
combined.xs('FRA', level='Country Code', drop_level=False).tail()

ENI         POP     URB           AFL         TI
Country Code Year                                                      
FRA          2015  0.08477  121.536704  79.655  43553.399292  24.493498
             2016      NaN  121.857825  79.917  43914.481647  24.306059
             2017      NaN  122.114008  80.180  44826.507474  24.298438
             2018      NaN  122.299437  80.444  45561.001768  24.224265
             2019      NaN         NaN  80.709  46183.521790        NaN

## Construct Squalli and Wilson (2011) CTS indicator

In [12]:
country_data = wdi_data.loc[is_country]
trade = country_data['NE.IMP.GNFS.CD'] + country_data['NE.EXP.GNFS.CD']
gdp = country_data['NY.GDP.MKTP.CD']
TS = trade / gdp
combined['TS'] = TS
TS.loc[pd.IndexSlice[['USA'], 1999:2001]]

Country Code  Year
USA           1999    0.232735
              2000    0.250436
              2001    0.228430
dtype: float64

In [13]:
world_trade = trade.groupby(level='Year').sum()
WTS = trade.groupby(level='Country Code').transform(lambda x: x.divide(world_trade))
WTS.loc[pd.IndexSlice[['USA'], 1999:2001]]

Country Code  Year
USA           1999    0.163413
              2000    0.168332
              2001    0.161606
dtype: float64

In [14]:
WTS_mean = WTS.groupby(level='Year').mean()
WTS_mean.loc[1999:2001]

Year
1999    0.005848
2000    0.005682
2001    0.005682
dtype: float64

In [15]:
relative_WTS = WTS.divide(WTS_mean)
relative_WTS.loc[pd.IndexSlice[['USA'], 1999:2001]]

Country Code  Year
USA           1999    27.943636
              2000    29.626372
              2001    28.442691
dtype: float64

In [16]:
CTS = relative_WTS.multiply(TS)
combined['CTS'] = CTS
CTS.loc[pd.IndexSlice[['USA'], 1999:2001]]

Country Code  Year
USA           1999    6.503455
              2000    7.419520
              2001    6.497152
dtype: float64

In [17]:
CTS.xs(2000, level='Year').dropna().sort_values().tail(10)

Country Code
FRA     4.904961
GBR     5.198786
MYS     5.257304
BEL     5.513525
CAN     5.906451
USA     7.419520
NLD     7.570988
DEU     8.487531
HKG    12.148940
SGP    14.717636
dtype: float64

# Add Chinn-Ito capital openness index

In [18]:
combined['KAOPEN'] = chinn_ito_data['KAOPEN']
combined['KAOPEN'].loc[pd.IndexSlice[['ROU'], 2000:2010]]

Country Code  Year
ROU           2000   -1.218540
              2001   -1.218540
              2002   -0.014185
              2003    0.241338
              2004    1.567014
              2005    1.822538
              2006    2.078061
              2007    2.333585
              2008    2.333585
              2009    2.333585
              2010    2.333585
Name: KAOPEN, dtype: float64

# Save out combined dataset

In [19]:
combined.xs(2000, level='Year')

,ENI,POP,URB,AFL,TI,TS,CTS,KAOPEN
Country Code,,,,,,,,
ABW,NaN,504.738889,46.717,41022.321554,NaN,1.450732,0.045496,-1.218540
AFG,NaN,31.829110,22.078,NaN,NaN,NaN,NaN,-1.920278
AGO,0.093161,13.151097,50.087,4707.441555,NaN,1.525471,0.245142,-1.218540
ALB,0.096493,112.738212,41.741,6015.897444,2.007105,0.634541,0.016170,-0.148388
AND,NaN,139.127660,92.395,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
XKX,0.192685,156.149536,NaN,4715.502918,NaN,NaN,NaN,NaN
YEM,NaN,32.973601,26.267,NaN,0.166580,NaN,NaN,2.333585
ZAF,0.240353,37.068732,56.891,10088.853369,7.327480,0.514378,0.416307,-1.218540


In [20]:
combined.to_csv(output_path)